In [9]:
import pandas as pd
import numpy as np

In [2]:
price_data = pd.read_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Dados_bolsa.xlsx', sheet_name= "Sheet1")
price_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.297814,985,ABEV3.SA
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.282718,227,ABEV3.SA
2,2000-01-07,0.494478,0.494478,0.494478,0.494478,0.282718,151,ABEV3.SA
3,2000-01-10,0.494478,0.494478,0.494478,0.494478,0.282718,1516,ABEV3.SA
4,2000-01-11,0.494478,0.494478,0.494478,0.494478,0.282718,3791,ABEV3.SA


In [5]:
price_data = price_data.rename(columns={'Ticker':'symbol', 'Date':'datetime','Close':'close','High':'high','Low':'low','Open':'open', 'Volume':'volume'})

price_data = price_data[['symbol','datetime','close','high','low','open','volume']]

# sort the values by symbol and then date
price_data.sort_values(by = ['symbol','datetime'], inplace = True)

# calculate the change in price
price_data['change_in_price'] = price_data['close'].diff()
price_data.head(10)

,symbol,datetime,close,high,low,open,volume,change_in_price
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN
1,ABEV3.SA,2000-01-06,0.494478,0.494478,0.494478,0.494478,227,-0.026404
2,ABEV3.SA,2000-01-07,0.494478,0.494478,0.494478,0.494478,151,0.000000
3,ABEV3.SA,2000-01-10,0.494478,0.494478,0.494478,0.494478,1516,0.000000
4,ABEV3.SA,2000-01-11,0.494478,0.494478,0.494478,0.494478,3791,0.000000
5,ABEV3.SA,2000-01-12,0.481293,0.481293,0.481293,0.481293,1137,-0.013185
6,ABEV3.SA,2000-01-13,0.484589,0.484589,0.484589,0.484589,606,0.003296
7,ABEV3.SA,2000-01-14,0.494478,0.494478,0.494478,0.494478,6445,0.009889
8,ABEV3.SA,2000-01-17,0.527474,0.527474,0.527474,0.527474,25556,0.032996
9,ABEV3.SA,2000-01-18,0.527474,0.527474,0.527474,0.527474,0,0.000000


In [6]:
price_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18643 entries, 0 to 18642
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   symbol           18643 non-null  object        
 1   datetime         18643 non-null  datetime64[ns]
 2   close            18643 non-null  float64       
 3   high             18643 non-null  float64       
 4   low              18643 non-null  float64       
 5   open             18643 non-null  float64       
 6   volume           18643 non-null  int64         
 7   change_in_price  18642 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 1.1+ MB


In [13]:
price_data.tail()

,symbol,datetime,close,high,low,open,volume,change_in_price
18638,VALE3.SA,2024-09-23,57.529999,57.860001,56.599998,57.090000,18164700,0.180000
18639,VALE3.SA,2024-09-24,60.340000,60.740002,59.639999,60.209999,44910100,2.810001
18640,VALE3.SA,2024-09-25,60.610001,61.250000,60.419998,60.720001,21557700,0.270000
18641,VALE3.SA,2024-09-26,64.250000,64.379997,62.450001,62.599998,61449300,3.639999
18642,VALE3.SA,2024-09-27,63.959999,65.040001,63.169998,64.699997,37237900,-0.290001


In [7]:
price_data1 = price_data.copy()

#### Tirando a diferença do 'close' na virada de um tick para outro.

In [16]:
# identify rows where the symbol changes
mask = price_data['symbol'] != price_data['symbol'].shift(1)

# For those rows, let's make the value null
price_data['change_in_price'] = np.where(mask == True, np.nan, price_data['change_in_price'])

# print the rows that have a null value, should only be 5
price_data[price_data.isna().any(axis = 1)]

,symbol,datetime,close,high,low,open,volume,change_in_price
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN
6213,SBSP3.SA,2000-01-03,7.815000,7.962916,7.777916,7.888750,2536,NaN
12428,VALE3.SA,2000-01-03,3.500000,3.542500,3.500000,3.500000,585600,NaN


In [24]:
price_data[mask]

,symbol,datetime,close,high,low,open,volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,0
6213,SBSP3.SA,2000-01-03,7.815000,7.962916,7.777916,7.888750,2536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.061137,NaN,0
12428,VALE3.SA,2000-01-03,3.500000,3.542500,3.500000,3.500000,585600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.097776,NaN,0


#### Calculando o RSI.

**Fórmula:**

\begin{align}
\ RSI = 100 - \frac{100}{1+RS}
\end{align}

In [17]:
# Calculate the 14 day RSI
n = 14

# First make a copy of the data frame twice
up_df, down_df = price_data[['symbol','change_in_price']].copy(), price_data[['symbol','change_in_price']].copy()

# For up days, if the change is less than 0 set to 0.
up_df.loc['change_in_price'] = up_df.loc[(up_df['change_in_price'] < 0), 'change_in_price'] = 0

# For down days, if the change is greater than 0 set to 0.
down_df.loc['change_in_price'] = down_df.loc[(down_df['change_in_price'] > 0), 'change_in_price'] = 0

# We need change in price to be absolute.
down_df['change_in_price'] = down_df['change_in_price'].abs()

# Calculate the EWMA (Exponential Weighted Moving Average), meaning older values are given less weight compared to newer values.
ewma_up = up_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())
ewma_down = down_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())

# Calculate the Relative Strength
relative_strength = ewma_up / ewma_down

# Calculate the Relative Strength Index
relative_strength_index = 100.0 - (100.0 / (1.0 + relative_strength))

# Add the info to the data frame.
price_data['down_days'] = down_df['change_in_price']
price_data['up_days'] = up_df['change_in_price']
price_data['RSI'] = relative_strength_index

# Display the head.
price_data.head(30)

,symbol,datetime,close,high,low,open,volume,change_in_price,down_days,up_days,RSI
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN,NaN,NaN,NaN
1,ABEV3.SA,2000-01-06,0.494478,0.494478,0.494478,0.494478,227,-0.026404,0.026404,0.000000,0.000000
2,ABEV3.SA,2000-01-07,0.494478,0.494478,0.494478,0.494478,151,0.000000,0.000000,0.000000,0.000000
3,ABEV3.SA,2000-01-10,0.494478,0.494478,0.494478,0.494478,1516,0.000000,0.000000,0.000000,0.000000
4,ABEV3.SA,2000-01-11,0.494478,0.494478,0.494478,0.494478,3791,0.000000,0.000000,0.000000,0.000000
5,ABEV3.SA,2000-01-12,0.481293,0.481293,0.481293,0.481293,1137,-0.013185,0.013185,0.000000,0.000000
6,ABEV3.SA,2000-01-13,0.484589,0.484589,0.484589,0.484589,606,0.003296,0.000000,0.003296,11.927770
7,ABEV3.SA,2000-01-14,0.494478,0.494478,0.494478,0.494478,6445,0.009889,0.000000,0.009889,37.666643
8,ABEV3.SA,2000-01-17,0.527474,0.527474,0.527474,0.527474,25556,0.032996,0.000000,0.032996,70.668652
9,ABEV3.SA,2000-01-18,0.527474,0.527474,0.527474,0.527474,0,0.000000,0.000000,0.000000,70.668652


## Calculando o Stochastic Oscillator

**Fórmula:**

\begin{align}
\ K = 100 \ast  \frac{(C-L_{14})}{(H_{14}-L_{14})}
\end{align}

$
\begin{align}
\ where,
\end{align}
$

$
\begin{align}
\ C = \text{Current Closing Price}
\end{align}
$

$
\begin{align}
\ L_{14} = \text{Lowest Low over the past 14 days}
\end{align}
$

$
\begin{align}
\ H_{14} = \text{Highest High over the past 14 days}
\end{align}
$


In [18]:
# Calculate the Stochastic Oscillator
n = 14

# Make a copy of the high and low column.
low_14, high_14 = price_data[['symbol','low']].copy(), price_data[['symbol','high']].copy()

# Group by symbol, then apply the rolling function and grab the Min and Max.
low_14 = low_14.groupby('symbol')['low'].transform(lambda x: x.rolling(window = n).min())
high_14 = high_14.groupby('symbol')['high'].transform(lambda x: x.rolling(window = n).max())

# Calculate the Stochastic Oscillator.
k_percent = 100 * ((price_data['close'] - low_14) / (high_14 - low_14))

# Add the info to the data frame.
price_data['low_14'] = low_14
price_data['high_14'] = high_14
price_data['k_percent'] = k_percent

# Display the head.
price_data.head(30)

,symbol,datetime,close,high,low,open,volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABEV3.SA,2000-01-06,0.494478,0.494478,0.494478,0.494478,227,-0.026404,0.026404,0.000000,0.000000,NaN,NaN,NaN
2,ABEV3.SA,2000-01-07,0.494478,0.494478,0.494478,0.494478,151,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
3,ABEV3.SA,2000-01-10,0.494478,0.494478,0.494478,0.494478,1516,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
4,ABEV3.SA,2000-01-11,0.494478,0.494478,0.494478,0.494478,3791,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
5,ABEV3.SA,2000-01-12,0.481293,0.481293,0.481293,0.481293,1137,-0.013185,0.013185,0.000000,0.000000,NaN,NaN,NaN
6,ABEV3.SA,2000-01-13,0.484589,0.484589,0.484589,0.484589,606,0.003296,0.000000,0.003296,11.927770,NaN,NaN,NaN
7,ABEV3.SA,2000-01-14,0.494478,0.494478,0.494478,0.494478,6445,0.009889,0.000000,0.009889,37.666643,NaN,NaN,NaN
8,ABEV3.SA,2000-01-17,0.527474,0.527474,0.527474,0.527474,25556,0.032996,0.000000,0.032996,70.668652,NaN,NaN,NaN
9,ABEV3.SA,2000-01-18,0.527474,0.527474,0.527474,0.527474,0,0.000000,0.000000,0.000000,70.668652,NaN,NaN,NaN


## Calculando o indicador Williams %R

**Fórmula:**

\begin{align}
\ R = \frac{(H_{14}-C)}{(H_{14}-L_{14})} * - 100
\end{align}

$
\begin{align}
\ where,
\end{align}
$

$
\begin{align}
\ C = \text{Current Closing Price}
\end{align}
$

$
\begin{align}
\ L_{14} = \text{Lowest Low over the past 14 days}
\end{align}
$

$
\begin{align}
\ H_{14} = \text{Highest High over the past 14 days}
\end{align}
$


In [20]:
# Calculate the Williams %R
n = 14

# Make a copy of the high and low column.
low_14, high_14 = price_data[['symbol','low']].copy(), price_data[['symbol','high']].copy()

# Group by symbol, then apply the rolling function and grab the Min and Max.
low_14 = low_14.groupby('symbol')['low'].transform(lambda x: x.rolling(window = n).min())
high_14 = high_14.groupby('symbol')['high'].transform(lambda x: x.rolling(window = n).max())

# Calculate William %R indicator.
r_percent = ((high_14 - price_data['close']) / (high_14 - low_14)) * (- 100)

# Add the info to the data frame.
price_data['r_percent'] = r_percent

# Display the head.
price_data.head(30)

,symbol,datetime,close,high,low,open,volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABEV3.SA,2000-01-06,0.494478,0.494478,0.494478,0.494478,227,-0.026404,0.026404,0.000000,0.000000,NaN,NaN,NaN,NaN
2,ABEV3.SA,2000-01-07,0.494478,0.494478,0.494478,0.494478,151,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
3,ABEV3.SA,2000-01-10,0.494478,0.494478,0.494478,0.494478,1516,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
4,ABEV3.SA,2000-01-11,0.494478,0.494478,0.494478,0.494478,3791,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
5,ABEV3.SA,2000-01-12,0.481293,0.481293,0.481293,0.481293,1137,-0.013185,0.013185,0.000000,0.000000,NaN,NaN,NaN,NaN
6,ABEV3.SA,2000-01-13,0.484589,0.484589,0.484589,0.484589,606,0.003296,0.000000,0.003296,11.927770,NaN,NaN,NaN,NaN
7,ABEV3.SA,2000-01-14,0.494478,0.494478,0.494478,0.494478,6445,0.009889,0.000000,0.009889,37.666643,NaN,NaN,NaN,NaN
8,ABEV3.SA,2000-01-17,0.527474,0.527474,0.527474,0.527474,25556,0.032996,0.000000,0.032996,70.668652,NaN,NaN,NaN,NaN
9,ABEV3.SA,2000-01-18,0.527474,0.527474,0.527474,0.527474,0,0.000000,0.000000,0.000000,70.668652,NaN,NaN,NaN,NaN


## Cálculo do indicador Moving Average Convergence Divergnece (MACD)

**Fórmula:**


\begin{align}
\ MACD = EMA_{12}(C) - EMA_{26}(C)
\end{align}

\begin{align}
\ SignalLine = EMA_{9}(MACD)
\end{align}


$
\begin{align}
\ where,
\end{align}
$

$
\begin{align}
\ MACD = \text{Moving Average Convergence Divergence}
\end{align}
$

$
\begin{align}
\ C = \text{Closing Price}
\end{align}
$

$
\begin{align}
\ EMA_{n} = \text{n day Exponential Moving Average}
\end{align}
\\
$


In [21]:
# Calculate the MACD
ema_26 = price_data.groupby('symbol')['close'].transform(lambda x: x.ewm(span = 26).mean())
ema_12 = price_data.groupby('symbol')['close'].transform(lambda x: x.ewm(span = 12).mean())
macd = ema_12 - ema_26

# Calculate the EMA
ema_9_macd = macd.ewm(span = 9).mean()

# Store the data in the data frame.
price_data['MACD'] = macd
price_data['MACD_EMA'] = ema_9_macd

# Print the head.
price_data.head(30)

,symbol,datetime,close,high,low,open,volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
1,ABEV3.SA,2000-01-06,0.494478,0.494478,0.494478,0.494478,227,-0.026404,0.026404,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000592,-0.000329
2,ABEV3.SA,2000-01-07,0.494478,0.494478,0.494478,0.494478,151,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000755,-0.000504
3,ABEV3.SA,2000-01-10,0.494478,0.494478,0.494478,0.494478,1516,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000810,-0.000607
4,ABEV3.SA,2000-01-11,0.494478,0.494478,0.494478,0.494478,3791,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000823,-0.000672
5,ABEV3.SA,2000-01-12,0.481293,0.481293,0.481293,0.481293,1137,-0.013185,0.013185,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.001379,-0.000863
6,ABEV3.SA,2000-01-13,0.484589,0.484589,0.484589,0.484589,606,0.003296,0.000000,0.003296,11.927770,NaN,NaN,NaN,NaN,-0.001563,-0.001041
7,ABEV3.SA,2000-01-14,0.494478,0.494478,0.494478,0.494478,6445,0.009889,0.000000,0.009889,37.666643,NaN,NaN,NaN,NaN,-0.001191,-0.001077
8,ABEV3.SA,2000-01-17,0.527474,0.527474,0.527474,0.527474,25556,0.032996,0.000000,0.032996,70.668652,NaN,NaN,NaN,NaN,0.000722,-0.000661
9,ABEV3.SA,2000-01-18,0.527474,0.527474,0.527474,0.527474,0,0.000000,0.000000,0.000000,70.668652,NaN,NaN,NaN,NaN,0.002069,-0.000049


### Cálculo do indicador Price Rate Of Change

**Formula:**


\begin{align}
\\
\ PROC_{t} = \frac{C_{t} - C_{t-n}}{C_{t-n}}
\\
\end{align}


$
\begin{align}
\ where,
\end{align}
$

$
\begin{align}
\ PROC_{t} = \text{Price Rate of Change at time t}
\end{align}
$

$
\begin{align}
\ C_{t} = \text{Closing price at time t}
\end{align}
\\
$


In [22]:
# Calculate the Price Rate of Change
n = 9

# Calculate the Rate of Change in the Price, and store it in the Data Frame.
price_data['Price_Rate_Of_Change'] = price_data.groupby('symbol')['close'].transform(lambda x: x.pct_change(periods = n))

# Print the first 30 rows
price_data.head(30)

,symbol,datetime,close,high,low,open,volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN
1,ABEV3.SA,2000-01-06,0.494478,0.494478,0.494478,0.494478,227,-0.026404,0.026404,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000592,-0.000329,NaN
2,ABEV3.SA,2000-01-07,0.494478,0.494478,0.494478,0.494478,151,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000755,-0.000504,NaN
3,ABEV3.SA,2000-01-10,0.494478,0.494478,0.494478,0.494478,1516,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000810,-0.000607,NaN
4,ABEV3.SA,2000-01-11,0.494478,0.494478,0.494478,0.494478,3791,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000823,-0.000672,NaN
5,ABEV3.SA,2000-01-12,0.481293,0.481293,0.481293,0.481293,1137,-0.013185,0.013185,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.001379,-0.000863,NaN
6,ABEV3.SA,2000-01-13,0.484589,0.484589,0.484589,0.484589,606,0.003296,0.000000,0.003296,11.927770,NaN,NaN,NaN,NaN,-0.001563,-0.001041,NaN
7,ABEV3.SA,2000-01-14,0.494478,0.494478,0.494478,0.494478,6445,0.009889,0.000000,0.009889,37.666643,NaN,NaN,NaN,NaN,-0.001191,-0.001077,NaN
8,ABEV3.SA,2000-01-17,0.527474,0.527474,0.527474,0.527474,25556,0.032996,0.000000,0.032996,70.668652,NaN,NaN,NaN,NaN,0.000722,-0.000661,NaN
9,ABEV3.SA,2000-01-18,0.527474,0.527474,0.527474,0.527474,0,0.000000,0.000000,0.000000,70.668652,NaN,NaN,NaN,NaN,0.002069,-0.000049,0.012655


## Cálculo do indicador On Balance Volume

#### Formula:

$
\begin{equation}
OBV (t) =
    \begin{cases}
      \text{OBV(t - 1) + Vol(t) if C(t) > C(t-1)}\\
      \text{OBV(t - 1) - Vol(t) if C(t) < C(t-1)}\\
      \text{OBV(t - 1) if C(t) = C(t-1)}\\
    \end{cases}\\
\\
\text{where,}\\
\\
\text{OBV (t) = on balance volume at time t}\\
\\
\text{Vol(t) = trading volume at time t}\\
\\
\text{C(t) = closing price at time t}\\
\end{equation}
$


In [23]:
def obv(group):

    # Grab the volume and close column.
    volume = group['volume']
    change = group['close'].diff()

    # intialize the previous OBV
    prev_obv = 0
    obv_values = []

    # calculate the On Balance Volume
    for i, j in zip(change, volume):

        if i > 0:
            current_obv = prev_obv + j
        elif i < 0:
            current_obv = prev_obv - j
        else:
            current_obv = prev_obv

        # OBV.append(current_OBV)
        prev_obv = current_obv
        obv_values.append(current_obv)
    
    # Return a panda series.
    return pd.Series(obv_values, index = group.index)
        

# apply the function to each group
obv_groups = price_data.groupby('symbol').apply(obv)

# add to the data frame, but drop the old index, before adding it.
price_data['On Balance Volume'] = obv_groups.reset_index(level=0, drop=True)

# display the data frame.
price_data.head(30)

,symbol,datetime,close,high,low,open,volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0
1,ABEV3.SA,2000-01-06,0.494478,0.494478,0.494478,0.494478,227,-0.026404,0.026404,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000592,-0.000329,NaN,-227
2,ABEV3.SA,2000-01-07,0.494478,0.494478,0.494478,0.494478,151,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000755,-0.000504,NaN,-227
3,ABEV3.SA,2000-01-10,0.494478,0.494478,0.494478,0.494478,1516,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000810,-0.000607,NaN,-227
4,ABEV3.SA,2000-01-11,0.494478,0.494478,0.494478,0.494478,3791,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000823,-0.000672,NaN,-227
5,ABEV3.SA,2000-01-12,0.481293,0.481293,0.481293,0.481293,1137,-0.013185,0.013185,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.001379,-0.000863,NaN,-1364
6,ABEV3.SA,2000-01-13,0.484589,0.484589,0.484589,0.484589,606,0.003296,0.000000,0.003296,11.927770,NaN,NaN,NaN,NaN,-0.001563,-0.001041,NaN,-758
7,ABEV3.SA,2000-01-14,0.494478,0.494478,0.494478,0.494478,6445,0.009889,0.000000,0.009889,37.666643,NaN,NaN,NaN,NaN,-0.001191,-0.001077,NaN,5687
8,ABEV3.SA,2000-01-17,0.527474,0.527474,0.527474,0.527474,25556,0.032996,0.000000,0.032996,70.668652,NaN,NaN,NaN,NaN,0.000722,-0.000661,NaN,31243
9,ABEV3.SA,2000-01-18,0.527474,0.527474,0.527474,0.527474,0,0.000000,0.000000,0.000000,70.668652,NaN,NaN,NaN,NaN,0.002069,-0.000049,0.012655,31243


## Modelando.

In [ ]:
# Create a column we wish to predict
'''
    In this case, let's create an output column that will be 1 if the closing price at time 't' is greater than 't-1' and 0 otherwise.
    In other words, if the today's closing price is greater than yesterday's closing price it would be 1.
'''

# Group by the `Symbol` column, then grab the `Close` column.
close_groups = price_data.groupby('symbol')['close']

# Apply the lambda function which will return -1.0 for down, 1.0 for up and 0.0 for no change.
close_groups = close_groups.transform(lambda x : np.sign(x.diff()))

# add the data to the main dataframe.
price_data['Prediction'] = close_groups

# for simplicity in later sections I'm going to make a change to our prediction column. To keep this as a binary classifier I'll change flat days and consider them up days.
price_data.loc[price_data['Prediction'] == 0.0] = 1.0

# print the head
price_data.head(50)

# OPTIONAL CODE: Dump the data frame to a CSV file to examine the data yourself.
# price_data.to_csv('final_metrics.csv')